<a href="https://colab.research.google.com/github/Siddhartha96123/Prot-Conf-Het/blob/main/Classic_Conf_Het.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### WELCOME to the ***CLASSIC*** Executable **Conformational Heterogeneity Scanner**.

It is capable of performing an ***'APO vs Holo'*** conformational heterogeneity scanning of same enzyme.

Please Press Play.

We need to first Install the Dependencies

In [ ]:
# Required libraries installation
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.8 MB/s eta 0:00:00


Now that the dependencies are done : Lets move onto the actual script.

The script will at the tail end of this code - ask for the **Reference PDB** first - which shall be needed to be uploaded from your local system, followed by the **Target PDB**.

Green tick on the left side of the Play button implies this part has been executed completely without any errors. Herein, we are :

1. Defining the function for uploading files and Parsing. Initalizing the ***Superimposer***.

2. Defining the function for aligning the structures.

3. Defining the function for calculating *RMSD/Res*

4. Defining the function to map the values on a an excel sheet.

5. Defining the function for plotting the values from the excel to a Graph.

In [ ]:
from Bio.PDB import PDBParser, Superimposer, is_aa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import os

# Function to upload files in Colab
def upload_files():
    uploaded = files.upload()
    file_paths = list(uploaded.keys())
    return file_paths

def align_structures(reference_path, target_path):
    parser = PDBParser(QUIET=True)

    # Load the reference structure
    reference_structure = parser.get_structure("reference", reference_path)

    # Load the target structure
    target_structure = parser.get_structure("target", target_path)

    # Check if structures are empty
    if len(reference_structure) == 0 or len(target_structure) == 0:
        raise ValueError("One or both of the structures are empty.")

    # Initialize the superimposer
    superimposer = Superimposer()
    reference_atoms = []
    target_atoms = []

    # Debugging: Print chain IDs to check if they match
    print("Reference chains:", [chain.id for chain in reference_structure[0]])
    print("Target chains:", [chain.id for chain in target_structure[0]])

    # Iterate over residues in the reference structure and select only CA atoms
    for ref_chain in reference_structure[0]:
        for ref_residue in ref_chain:
            if is_aa(ref_residue):  # Check if residue is an amino acid
                try:
                    ref_atom = ref_residue['CA']  # Select C-alpha atom

                    # Try to find the closest residue in the target structure based on sequence
                    for tgt_chain in target_structure[0]:
                        try:
                            # Match based on position in the sequence (not residue ID)
                            tgt_residue = tgt_chain[ref_residue.id]
                            tgt_atom = tgt_residue['CA']  # Select C-alpha atom

                            # Append atoms for alignment only if both CA atoms exist
                            reference_atoms.append(ref_atom)
                            target_atoms.append(tgt_atom)

                        except KeyError:
                            continue  # Skip residues not found in the target structure
                except KeyError:
                    print(f"Skipping atom in residue {ref_residue} (missing CA atom in reference).")
                    continue

    # Check if atoms are available for superimposition
    if not reference_atoms or not target_atoms:
        raise ValueError("No atoms available for reference superimposition.")

    # Perform the superimposition
    superimposer.set_atoms(reference_atoms, target_atoms)
    superimposer.apply(target_structure[0].get_atoms())

    return reference_structure, target_structure

def calculate_distances(reference_structure, target_structure):
    distances = []

    # Iterate over residues and calculate distances only for CA atoms
    for ref_chain in reference_structure[0]:
        for ref_residue in ref_chain:
            if is_aa(ref_residue):  # Check if residue is an amino acid
                try:
                    ref_atom = ref_residue['CA']  # Select C-alpha atom
                    for tgt_chain in target_structure[0]:
                        try:
                            tgt_residue = tgt_chain[ref_residue.id]
                            tgt_atom = tgt_residue['CA']  # Select C-alpha atom
                            distance = np.linalg.norm(ref_atom.get_coord() - tgt_atom.get_coord())
                            distances.append((ref_residue.id[1], tgt_residue.id[1], distance))  # Use residue ID
                        except KeyError:
                            # Skip residues not found in the target structure
                            continue
                except KeyError:
                    # Skip atoms not found in the reference structure
                    continue

    return distances

def save_distances_to_excel(distances, output_path):
    df = pd.DataFrame(distances, columns=["Reference Residue", "Aligned Residue", "Distance"])
    df.to_excel(output_path, index=False)

def plot_rmsf(distances, output_plot_path):
    reference_residues = [d[0] for d in distances]
    rmsf_values = [d[2] for d in distances]

    plt.scatter(reference_residues, rmsf_values)
    plt.xlabel("Reference Residue")
    plt.ylabel("RMSF (Å)")
    plt.title("Root Mean Square Fluctuation")
    plt.savefig(output_plot_path)
    plt.close()

The green tick on the above block here signals that the functions to perform every associated funciton within this tool have been ***Defined & Initialised***  

We can finally now, move to uploading the ***SINGLE CHAIN PDBs*** for all further analyses.

In [ ]:
# Upload the PDB files individually
print("Please upload the reference PDB file.")
reference_pdb = upload_files()[0]

print("Please upload the target PDB file.")
target_pdb = upload_files()[0]

try:
    # Align structures
    reference_structure, aligned_structure = align_structures(reference_pdb, target_pdb)

    # Calculate distances between atoms
    distances = calculate_distances(reference_structure, aligned_structure)

    # Save distances to Excel
    output_excel = "temp_PDB_RMSF.xlsx"
    save_distances_to_excel(distances, output_excel)

    # Plot RMSF
    output_plot = "temp_PDB_RMSF_plot.png"
    plot_rmsf(distances, output_plot)

    # Download the files
    print("Script execution completed. \n The temp Excel file and the Plot shall be auto-downloaded to your local system now. \n Please allow permissions.")
    files.download(output_excel)
    files.download(output_plot)

except Exception as e:
    print("An error occurred during script execution:", str(e))

Please upload the reference PDB file.


Saving PfKRS-KSA-x22_1-P21212-25A-195Apr23_D.pdb to PfKRS-KSA-x22_1-P21212-25A-195Apr23_D (2).pdb
Please upload the target PDB file.


Saving 6chd_chain-a.pdb to 6chd_chain-a (2).pdb
Reference chains: ['D']
Target chains: ['A']
Script execution completed. 
 The temp Excel file and the Plot shall be auto-downloaded to your local system now. 
 Please allow permissions.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ***MAZE KARO.***